<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/New/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%96%E2%80%AF11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Практическая работа № 11. Обучение с подкреплением для управления

## 1. Цель и задачи работы

**Цель работы** — формирование у обучающегося системного представления о методологии применения алгоритмов обучения с подкреплением к задачам управления реальными и симулированными динамическими системами. Работа направлена на освоение практических навыков проектирования сред, формулировки функций вознаграждения, выбора архитектур агентов, а также на проведение строгой оценки качества управления по критериям стабильности, эффективности и обобщающей способности.

**Основные задачи работы**:

- Использовать **две управляющие среды** различной сложности:
  – **Низкоразмерная среда**: классическая задача управления (например, *Inverted Pendulum*, *MountainCar*, *Lunar Lander*) с непрерывным или дискретным пространством действий;
  – **Высокоразмерная среда**: прикладная задача управления, основанная на данных из Практической работы № 1 (например, управление температурой в здании, балансировка энергосистемы, оптимизация запасов), где:
    • состояние включает временные ряды (≥720 наблюдений) и инженерные признаки (лаги, скользящие средние, циклические кодировки);
    • действия — дискретные или непрерывные управляющие сигналы (например, включение/выключение оборудования, регулировка мощности);
    • динамика среды — либо симулирована на основе физических моделей, либо аппроксимирована с помощью генеративной модели (например, TimeGAN из Практической работы № 8).

- Формализовать задачу управления как **марковский процесс принятия решений (MDP)**:
  – определить пространство состояний \( \mathcal{S} \), действий \( \mathcal{A} \), функцию переходов \( P(s'|s,a) \) (явную или имитационную);
  – разработать **функцию вознаграждения** \( r(s,a) \), отражающую цели управления (минимизация энергопотребления, поддержание заданного уровня, избегание нарушений ограничений).

- Определить архитектуры и гиперпараметры для всех агентов и провести их обучение **только на тренировочной части данных**, без доступа к тестовой среде.

- Реализовать и сравнить следующие категории алгоритмов обучения с подкреплением:
  – **Для дискретных действий**:
    • DQN (Deep Q-Network) с приоритетным реплейем;
    • Double DQN, Dueling DQN;
  – **Для непрерывных действий**:
    • DDPG (Deep Deterministic Policy Gradient);
    • TD3 (Twin Delayed DDPG);
    • SAC (Soft Actor-Critic);
  – **Современные подходы**:
    • PPO (Proximal Policy Optimization) — для обеих сред;
    • QR-DQN / Implicit Quantile Networks — для робастности;
    • Model-Based RL (например, MBPO или PILCO), если доступна дифференцируемая модель среды.

- Обеспечить **безопасность обучения**:
  – введение штрафов за нарушение физических или операционных ограничений;
  – использование constrained RL (например, Lagrangian relaxation) при наличии жёстких ограничений.

- Сгенерировать последовательности управления:
  – для каждой среды — не менее 100 эпизодов оценки;
  – длина эпизода — сопоставима с исходными временными рядами (≥30 шагов).

- Провести углублённую диагностику качества управления по трём критериям:
  – **Эффективность**: среднее накопленное вознаграждение, энергозатраты, отклонение от целевого режима;
  – **Стабильность**: дисперсия вознаграждения по эпизодам, частота нарушений ограничений;
  – **Обобщающая способность**: качество управления на **тестовой части среды** (например, на данных за другой сезон или при изменённой динамике).

- Оценить стабильность и эффективность алгоритмов:
  – время обучения (в часах);
  – время генерации одного эпизода (в секундах);
  – устойчивость к осцилляции Q-значений, коллапсу политики, переобучению.

- Проанализировать интерпретируемость политики:
  – визуализация траекторий состояний и действий;
  – анализ attention-карт (если используется трансформерная политика);
  – чувствительность действий к изменениям в ключевых признаках (через SHAP или integrated gradients).

- Исследовать практические сценарии:
  – **Адаптивное управление**: реакция агента на внезапное изменение внешних условий (например, резкое похолодание);
  – **Оптимизация по нескольким целям**: баланс между стоимостью и комфортом через multi-objective RL;
  – **Имитация эксперта**: комбинирование RL с Imitation Learning (например, DAgger) при наличии демонстраций.

- Разработать **модульный RL-фреймворк** с единым интерфейсом:
  – `fit(env_train)` — обучение агента;
  – `act(state)` — выбор действия;
  – `evaluate(env_test)` — оценка на тестовой среде.

- Обеспечить воспроизводимость и открытость результатов:
  – опубликовать код, обученные политики, логи обучения, сравнительные таблицы и аналитический отчёт в публичном репозитории;
  – использовать открытые лицензии: MIT (код), CC BY 4.0 (данные и политики).

## 2. Теоретические предпосылки

Обучение с подкреплением решает задачу последовательного принятия решений в условиях неопределённости, где агент взаимодействует со средой, чтобы максимизировать ожидаемое дисконтированное вознаграждение. В отличие от контролируемых методов, RL не требует размеченных пар «вход–действие», а учится через пробу и ошибку, что делает его особенно подходящим для задач управления.

Ключевая сложность в прикладных задачах — **формулировка функции вознаграждения**. Неправильно спроектированная награда может привести к «хакингу» среды (reward hacking): агент достигает высокого вознаграждения, но не решает практическую задачу (например, выключает датчик вместо поддержания температуры). Поэтому в работе делается акцент на **спецификацию награды через компромисс между целями и ограничениями**.

Современные алгоритмы делятся на **model-free** (DQN, SAC, PPO) и **model-based** (MBPO, PILCO). Первые проще в реализации и устойчивы к ошибкам моделирования, но требуют большого числа взаимодействий. Вторые эффективнее по данным, но чувствительны к неточностям модели среды — особенно критичным при работе с реальными временными рядами.

Для задач управления с **ограничениями** (например, максимальная мощность, безопасные температуры) применяются методы **constrained RL**, где вознаграждение дополняется штрафами или задача переформулируется как оптимизация при ограничениях на ожидаемые затраты.

Важно подчеркнуть: цель RL в управлении — не максимизация вознаграждения «во что бы то ни стало», а **обеспечение надёжного, безопасного и энергоэффективного поведения** в реальных условиях. Агент считается успешным, если демонстрирует стабильное и близкое к оптимальному поведение на **невидимых данных** (например, зимой, если обучался летом).

## 3. Порядок выполнения работы

### 3.1. Подбор и настройка сред

Задача: Обеспечить реалистичные условия управления.  
Требования:
– Низкоразмерная среда — из Gymnasium/Gym;
– Высокоразмерная среда — на основе данных из Практической работы № 1:
  • Состояние: [температура, влажность, лаги, время суток, день недели];
  • Действие: дискретное (вкл/выкл) или непрерывное (мощность от 0 до 1);
  • Динамика: симулятор на основе TVAE или TimeGAN из ПР № 8;
– Разделение временного ряда: 80 % — обучение, 20 % — тест (другой сезон/период).

### 3.2. Формализация MDP и проектирование награды

Задача: Корректно сформулировать задачу RL.  
Требования:
– Пространство состояний — нормализовано по тренировочной части;
– Пространство действий — явно задано (дискретное или ограниченное непрерывное);
– Функция вознаграждения:
  • Основная компонента: минимизация отклонения от целевой температуры;
  • Штрафы: за переключения, нарушение ограничений, энергопотребление;
  • Вознаграждение не должно зависеть от внутренней структуры среды (no information leakage).

### 3.3. Определение архитектур и обучение агентов

Задача: Привести каждый алгоритм к рабочей конфигурации.  
Требования:
– Использовать реализации из `stable-baselines3`, `RLlib`, или воспроизведение по статьям;
– Гиперпараметры по умолчанию или из литературы:
  • DQN: lr=1e-4, buffer_size=50000, batch_size=64;
  • SAC: entropy_coef=0.2, gamma=0.99;
  • PPO: n_steps=2048, gae_lambda=0.95;
– Обучение — только на тренировочной части среды;
– Фиксация: время обучения, стабильность loss, частота нарушений ограничений.

### 3.4. Генерация управляющих траекторий

Задача: Получить сопоставимые последовательности управления.  
Требования:
– Для каждой среды — 100 эпизодов оценки;
– Длина эпизода — ≥30 шагов;
– Запись: состояния, действия, вознаграждения, флаги нарушений.

### 3.5. Построение и настройка агентов

Задача: Обеспечить единообразный интерфейс.  
Требования:
– Все агенты реализуют методы:
  • `fit(env_train)`;
  • `act(state)`;
  • `evaluate(env_test)`;
– Для MBPO/Constrained RL — кастомная реализация при отсутствии open-source версии.

### 3.6. Диагностика качества управления

Задача: Объективно оценить эффективность агентов.  
Требования:
– **Эффективность**:
  • Среднее суммарное вознаграждение;
  • Среднее абсолютное отклонение от цели;
  • Суммарное энергопотребление;
– **Стабильность**:
  • Стандартное отклонение вознаграждения по эпизодам;
  • Доля эпизодов с нарушением ограничений;
– **Обобщающая способность**:
  • Сравнение метрик на train vs. test средах;
  • Тест при изменённой динамике (например, удвоенное тепловыделение).

### 3.7. Оценка качества и сравнительный анализ

Задача: Ранжировать агентов по комплексному критерию.  
Требования:
– Составление сравнительных таблиц:
  • Алгоритм × Среда × Вознаграждение × Стабильность × Обобщение × Время обучения;
– Ранжирование по взвешенной оценке:
  • Эффективность (40 %),
  • Стабильность и безопасность (40 %),
  • Вычислительная эффективность (20 %).

### 3.8. Анализ интерпретируемости политики

Задача: Понять, «как» и «почему» агент принимает решения.  
Требования:
– Визуализация траекторий (состояние–действие–вознаграждение);
– Для нейросетевых политик — анализ градиентов/SHAP по ключевым признакам;
– Чувствительность: как изменится действие при ±10 % к температуре/лагу?

### 3.9. Анализ вычислительной эффективности

Задача: Оценить практическую применимость.  
Требования:
– Фиксация времени:
  • Обучение (часы),
  • Один шаг управления (мс);
– Оценка памяти (GPU/CPU);
– Классификация: низкая / средняя / высокая сложность.

### 3.10. Практические сценарии применения

Задача: Продемонстрировать ценность RL в реальных задачах.  
Требования:
– **Адаптация к сдвигу**: обучение летом, управление зимой — сравнение с ПИД-регулятором;
– **Multi-objective control**: минимизация энергии при поддержании комфорта (через взвешенную награду);
– **Hybrid control**: комбинация RL с правилами (например, «если температура > 30°C — включить максимум»).

### 3.11. Публикация результатов исследования

Задача: Обеспечить открытость и воспроизводимость.  
Требования:
– В репозитории:
  • Код фреймворка;
  • Обученные модели (.zip/.pth);
  • Логи обучения (TensorBoard или CSV);
  • Сравнительные таблицы и визуализации;
– Лицензии: MIT (код), CC BY 4.0 (данные и политики);
– README с инструкцией по воспроизведению.

## 4. Дополнительные исследовательские задания

- Сравнение **SAC** и **ПИД-регулятора** по энергоэффективности и устойчивости;
- Анализ **опасного поведения**: может ли агент нарушать ограничения в тестовой среде?
- Управление с **частичной наблюдаемостью** (POMDP): скрытие части состояния;
- **Перенос политики**: обучение на симуляторе → развёртывание на реальных данных;
- Исследование зависимости качества от **формы функции вознаграждения** (линейная, квадратичная, sparse);
- Оценка **CO₂-следа** обучения: сравнение DDPG и PPO по энергопотреблению.

## 5. Требования к отчёту

Отчёт оформляется в соответствии с **ГОСТ 7.32–2017** и должен содержать:

- **Введение** — постановка задачи управления, актуальность RL, обзор литературы (DQN, SAC, PPO), связь с Практическими работами № 1 и № 8;
- **Методология** — описание сред, MDP-формализация, функции вознаграждения, архитектур агентов, метрик качества;
- **Результаты экспериментов** — сравнительные таблицы, графики вознаграждения, траектории управления, визуализации политик;
- **Обсуждение** — анализ компромиссов между эффективностью, безопасностью и сложностью; сравнение RL и классических контроллеров;
- **Заключение** — итоговые выводы, рекомендации по выбору алгоритма, ограничения исследования;
- **Список использованных источников** — по ГОСТ Р 7.0.5–2008;
- **Приложения** — фрагменты кода, гиперпараметры, скриншоты сред, примеры траекторий.

Отчёт сопровождается ссылкой на публичный репозиторий с кодом и результатами.

## 6. Критерии оценивания

| Оценка | Критерии |
|--------|--------|
| **Отлично** | Реализованы ≥4 алгоритма RL; оценка по трём критериям (эффективность, стабильность, обобщение); безопасность управления; практические сценарии; модульный код; публикация; отчёт по ГОСТ. |
| **Хорошо** | Реализованы 2–3 алгоритма; есть сравнительные таблицы и базовая диагностика; отсутствует лишь отдельный элемент (например, анализ обобщения). |
| **Удовлетворительно** | Только DQN или SAC; есть траектории и простые метрики; нет углублённой оценки стабильности или безопасности. |
| **Неудовлетворительно** | Управление без оценки качества; нет разделения train/test; отсутствует сравнительный анализ; отчёт не представлен. |
